# Caso Conectividad de Viajes

Los supuestos utilizados para abordar este caso son los siguientes:
* Para los **OD's** posibles/potenciales se condieró una sola parada entre el origen y destino. (Ej: Buenos Aires - Santiago y Santiago - Lima)
* Se consideró que los horarios de salida corresponden a un único día, pero no necesariamente los horarios de llegada, en otras palabras, si el vuelo sale a las 20:00 hrs y llega a las 9:00 hrs, estaría llegando al día siguiente.

--------------------------


Antes de poder manejar los datos en Python primero es necesario exportarlos desde Excel a un csv. Una vez hecho esto es posible guardar los datos del csv en un data frame con el siguiente código:

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 80)
pd.set_option('display.min_rows', 60)

url = 'https://raw.githubusercontent.com/ClaudioSanz/CasoConectividadDeViajes/main/Datos%20Caso%20Conectividad%20de%20Viajes.csv'

data = pd.read_csv(url, delimiter = ";")

data.head(28)


,Origen,Destino,Hora Salida,Hora Llegada
0,Buenos Aires,Santiago,16:00,18:00
1,Santiago,Buenos Aires,22:30,0:30
2,Buenos Aires,Lima,7:30,12:30
3,Lima,Buenos Aires,12:00,17:00
4,Sao Paulo,Lima,8:30,13:30
5,Lima,Sao Paulo,16:30,21:30
6,Sao Paulo,Santiago,14:30,18:30
7,Santiago,Sao Paulo,19:30,23:30
8,Santiago,Lima,10:30,14:30
9,Lima,Santiago,15:00,19:00


Luego, se crean dos funciones de conversión de unidades:
* Una para convertir los datos de tiempo en valores enteros.
* Y otra para convertir los valores enteros en valores de tiempo.

In [2]:
def TimeToInt (t):
    Int = []
    for i in range(len(t)):
        (h, m) = t[i].split(':')
        result = int(h) * 60 + int(m)
        Int.append(result)
    return Int   

def IntToTime (Int):
    time = []
    for i in range(len(Int)):
        if (Int[i] > 1439):
            Int[i] -= 1440 
        h = str(Int[i]//60)
        if(Int[i]%60 < 10): 
            m = "0" + str(Int[i]%60)
        else:
            m = str(Int[i]%60)
        result = h + ":" + m
        time.append(result)
    return time    

Con estas funciones es posible convertir las columnas "Hora Salida" y "Hora Llegada" en columnas que contengan valores en unidad de minutos. También hay que considerar que para aquellos casos donde la hora de llegada corresponde al día siguiente se deben sumar 1440 minutos para efectos de cálculo.

In [3]:
data["Hora Salida Min"] = TimeToInt(data["Hora Salida"].values.tolist()) 
data["Hora Llegada Min"] = TimeToInt(data["Hora Llegada"].values.tolist())

for i in range(len(data)):
    if (data["Hora Llegada Min"][i] < data["Hora Salida Min"][i]):
        data.loc[[i],["Hora Llegada Min"]] += 60 * 24
data

,Origen,Destino,Hora Salida,Hora Llegada,Hora Salida Min,Hora Llegada Min
0,Buenos Aires,Santiago,16:00,18:00,960,1080
1,Santiago,Buenos Aires,22:30,0:30,1350,1470
2,Buenos Aires,Lima,7:30,12:30,450,750
3,Lima,Buenos Aires,12:00,17:00,720,1020
4,Sao Paulo,Lima,8:30,13:30,510,810
5,Lima,Sao Paulo,16:30,21:30,990,1290
6,Sao Paulo,Santiago,14:30,18:30,870,1110
7,Santiago,Sao Paulo,19:30,23:30,1170,1410
8,Santiago,Lima,10:30,14:30,630,870
9,Lima,Santiago,15:00,19:00,900,1140


Luego, para encontrar todos los **Origenes-Destinos (OD)** posibles que podría lograr este itinerario considerando una parada se siguen los siguientes pasos:

* Lo primero es crear una lista que contenga todos los orígenes posibles.
* Luego, se crea un diccionario de *Posibles Paradas* que contenga los nombres de las columnas como llaves y listas vacías como valores.
* Posteriormente, se crea un **for** que recorra los valores de los orígenes uno por uno.
* Dentro de ese **for** se crean 3 listas vacías para guardar los datos de la parada candidata. Además, dentro de éste **for** se crea otro **for** que recorra los datos del itinerario y que contenga un condicional **if** que evalúe si se está ubicado en el Origen[i], y de ser así guarde el destino como una *parada candidata* y la hora de salida y de llegada de ese vuelo.
* Seguido, se crea otro **for** dentro del primero que recorra nuevamente la data del itinerario y otro dentro de éste último que recorra las *paradas candidatas*, con un **if** para evaluar si el Origen[k] donde se está ubicado corresponde a una *parada candidata* y si esta satisface la condición de que la hora de salida sea menor a la hora de llegada de la *parada candidata*, de modo que si se satisfacen estas dos condiciones, se agreguen dichos valores al diccionario de *Posibles Paradas*.
* Finalmente, se guardan los datos del diccionario como un Data Frame llamado *PosiblesParadasDF* y se transforman los datos de minutos a tiempo nuevamente.

In [4]:
Origenes = data["Origen"].unique().tolist()

Origenes

['Buenos Aires',
 'Santiago',
 'Lima',
 'Sao Paulo',
 'Sydney',
 'Calama',
 'Isla de Pascua',
 'Miami',
 'Nueva York',
 'Cuzco',
 'Madrid',
 'Londres',
 'Rio']

In [5]:
PosiblesParadas = {
        "Origen": [],
        "Parada": [],
        "Destino": [],
        "Hora Salida 1": [],
        "Hora Llegada 1": [],
        "Hora Salida 2": [],
        "Hora Llegada 2": []
    }

for i in range(len(Origenes)):
    ParadasCandidatas = []
    HoraSalidaCandidatas = []
    HoraLlegadaCandidatas = []
    for j in range(len(data)):
        if (data["Origen"][j] == Origenes[i]):
            ParadasCandidatas.append(data["Destino"][j])
            HoraSalidaCandidatas.append(data["Hora Salida Min"][j])
            HoraLlegadaCandidatas.append(data["Hora Llegada Min"][j])
    for k in range(len(data)):
        for l in range(len(ParadasCandidatas)):
            if ((data["Origen"][k] == ParadasCandidatas[l]) and (data["Hora Salida Min"][k] > HoraLlegadaCandidatas[l])):
                PosiblesParadas["Origen"].append(Origenes[i])
                PosiblesParadas["Parada"].append(data["Origen"][k])
                PosiblesParadas["Destino"].append(data["Destino"][k])
                PosiblesParadas["Hora Salida 1"].append(HoraSalidaCandidatas[l])
                PosiblesParadas["Hora Llegada 1"].append(HoraLlegadaCandidatas[l])
                PosiblesParadas["Hora Salida 2"].append(data["Hora Salida Min"][k])
                PosiblesParadas["Hora Llegada 2"].append(data["Hora Llegada Min"][k])
                
PosiblesParadasDF = pd.DataFrame(PosiblesParadas)
PosiblesParadasDF["Hora Salida 1"] = IntToTime(PosiblesParadasDF["Hora Salida 1"].values.tolist()) 
PosiblesParadasDF["Hora Llegada 1"] = IntToTime(PosiblesParadasDF["Hora Llegada 1"].values.tolist())
PosiblesParadasDF["Hora Salida 2"] = IntToTime(PosiblesParadasDF["Hora Salida 2"].values.tolist())
PosiblesParadasDF["Hora Llegada 2"] = IntToTime(PosiblesParadasDF["Hora Llegada 2"].values.tolist())
PosiblesParadasDF


,Origen,Parada,Destino,Hora Salida 1,Hora Llegada 1,Hora Salida 2,Hora Llegada 2
0,Buenos Aires,Santiago,Buenos Aires,16:00,18:00,22:30,0:30
1,Buenos Aires,Lima,Sao Paulo,7:30,12:30,16:30,21:30
2,Buenos Aires,Santiago,Sao Paulo,16:00,18:00,19:30,23:30
3,Buenos Aires,Lima,Santiago,7:30,12:30,15:00,19:00
4,Buenos Aires,Santiago,Sydney,16:00,18:00,20:00,9:00
5,Buenos Aires,Santiago,Calama,16:00,18:00,20:30,22:30
6,Buenos Aires,Santiago,Isla de Pascua,16:00,18:00,20:15,1:15
7,Buenos Aires,Lima,Miami,7:30,12:30,16:00,22:00
8,Buenos Aires,Lima,Nueva York,7:30,12:30,18:00,2:00
9,Buenos Aires,Lima,Cuzco,7:30,12:30,17:00,18:00


Si se desea obtener un excel con los datos de los posibles/potenciales **Orígenes - Destino** (OD's) se puede ejecutar el siguiente código:

In [23]:
import os
cwd = os.getcwd().replace("\\","/")
cwd
PosiblesParadasDF.to_excel(cwd + "/Caso Conectividad de Viajes Resultado.xls")

Para el caso de encontrar los **OD´s** conectados por cierta aerolínea se deben seguir los mismos pasos del punto anterior, agregando en el segundo **if** la restricción de que el Destino[k] sea igual al Origen[i] que se esté iterando, y modificando la restricción de tiempo para considerar una hora como mínimo entre la hora de salida del vuelo y la hora de llegada de la parada candidata.

In [24]:
PosiblesParadas = {
        "Origen": [],
        "Parada": [],
        "Destino": [],
        "Hora Salida 1": [],
        "Hora Llegada 1": [],
        "Hora Salida 2": [],
        "Hora Llegada 2": []
    }

for i in range(len(Origenes)):
    ParadasCandidatas = []
    HoraSalidaCandidatas = []
    HoraLlegadaCandidatas = []
    for j in range(len(data)):
        if (data["Origen"][j] == Origenes[i]):
            ParadasCandidatas.append(data["Destino"][j])
            HoraSalidaCandidatas.append(data["Hora Salida Min"][j])
            HoraLlegadaCandidatas.append(data["Hora Llegada Min"][j])
    for k in range(len(data)):
        for l in range(len(ParadasCandidatas)):
            if ((data["Origen"][k] == ParadasCandidatas[l]) and (data["Destino"][k] == Origenes[i]) and ((data["Hora Salida Min"][k] - HoraLlegadaCandidatas[l]) >= 60)):
                PosiblesParadas["Origen"].append(Origenes[i])
                PosiblesParadas["Parada"].append(data["Origen"][k])
                PosiblesParadas["Destino"].append(data["Destino"][k])
                PosiblesParadas["Hora Salida 1"].append(HoraSalidaCandidatas[l])
                PosiblesParadas["Hora Llegada 1"].append(HoraLlegadaCandidatas[l])
                PosiblesParadas["Hora Salida 2"].append(data["Hora Salida Min"][k])
                PosiblesParadas["Hora Llegada 2"].append(data["Hora Llegada Min"][k])
                
PosiblesParadasDF2 = pd.DataFrame(PosiblesParadas)
PosiblesParadasDF2["Hora Salida 1"] = IntToTime(PosiblesParadasDF2["Hora Salida 1"].values.tolist()) 
PosiblesParadasDF2["Hora Llegada 1"] = IntToTime(PosiblesParadasDF2["Hora Llegada 1"].values.tolist())
PosiblesParadasDF2["Hora Salida 2"] = IntToTime(PosiblesParadasDF2["Hora Salida 2"].values.tolist())
PosiblesParadasDF2["Hora Llegada 2"] = IntToTime(PosiblesParadasDF2["Hora Llegada 2"].values.tolist())
PosiblesParadasDF2

,Origen,Parada,Destino,Hora Salida 1,Hora Llegada 1,Hora Salida 2,Hora Llegada 2
0,Buenos Aires,Santiago,Buenos Aires,16:00,18:00,22:30,0:30
1,Sao Paulo,Lima,Sao Paulo,8:30,13:30,16:30,21:30
2,Sao Paulo,Santiago,Sao Paulo,14:30,18:30,19:30,23:30
3,Sydney,Santiago,Sydney,0:00,13:00,20:00,9:00
4,Isla de Pascua,Santiago,Isla de Pascua,9:00,14:00,20:15,1:15
5,Miami,Lima,Miami,8:30,14:30,16:00,22:00
6,Nueva York,Lima,Nueva York,4:00,12:00,18:00,2:00
7,Cuzco,Lima,Cuzco,12:00,13:00,17:00,18:00
8,Madrid,Sao Paulo,Madrid,10:00,20:00,23:00,9:00
9,Londres,Sao Paulo,Londres,12:00,22:00,23:15,9:15


Si se desea obtener un excel con los datos de los **OD's** conectados por cierta aerolínea se puede ejecutar el siguiente código:

In [25]:
PosiblesParadasDF2.to_excel(cwd + "/Caso Conectividad de Viajes Resultado 2.xls")

Una vez obtenidos ambos Data Frames, es posible calcular el porcentaje de conectividad como la razón entre la cantidad de **OD's** conectados por cierta aerolínea y los **OD's** potenciales, resultando en un porcentaje de conectividad del 16,18%.

In [26]:
PorcentajeDeConectividad = len(PosiblesParadasDF2)/len(PosiblesParadasDF) * 100
PorcentajeDeConectividad

16.176470588235293

Finalmente, otros indicadores que podrían ser relevantes para medir la conectividad aérea de un itinerario son: 
1. Un indicador que mida la conectividad de la aerolínea respecto a latinoamérica. Con este indicador se podría comparar la conectividad que tiene cierta aerolínea frente a otras aerolíneas en latinoamérica y a su vez evaluar posibles alianzas con éstas para aumentar la conectividad.
2. Otro indicador que mida la frecuencia de viajes realizados hacia o desde las ciudades de cierto país. Con este indicador sería posible analizar cuales son los destinos que se eligen con mayor frecuencia o menor frecuencia respecto de cierto país y así evaluar un posible aumento o disminución de conectividad con ciudades de dicho país.
